# Set environment

In [1]:
source("config_path.R")

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,


# Import Data

In [2]:
attach(file.path(OUTDIR, "HTS-GroupAll-DESeq.RData"))

In [6]:
attach(file.path(OUTDIR, "HTS-GO-Info.RData"))

The following objects are masked from file:/shared_space/TA_clint/analysis_output/out/HTS-GO-Info.RData (pos = 3):

    dat_go_bp_annot, dat_go_cc_annot, dat_go_cne_h99, dat_go_fc_annot,
    dat_go_info_bp, dat_go_info_cc, dat_go_info_fc



In [10]:
gage_wrapper <- function(rld, gset, crt, trt){
    gage::gage(
        assay(rld),
        gsets = gset,
        use.fold = FALSE,
                      
        ### reference condition or phenotype 
        ### (i.e. the control group) 
        ref  = which(colData(rld)[["Group"]] == crt), 
    
        ### a numeric vector of column numbers for 
        ### the target condition or phenotype 
        ### (i.e.the experiment group)
        samp = which(colData(rld)[["Group"]] == trt),
        compare = "unpaired",
                        
        ### he effective gene set size, 
        ### i.e. the number of genes included in the gene set test
        set.size = c(10, 500),
                      
        ### provide two sided test "greater" & "less"
        same.dir = TRUE)
} # end func

# 01 YPD -> TC in H99

In [17]:
gageres_media_h99 <- list()
gageres_media_h99$bp <- gage_wrapper(rlgexp, gset_go_bp, "YPDH99", "TCH99")
gageres_media_h99$fc <- gage_wrapper(rlgexp, gset_go_fc, "YPDH99", "TCH99")
gageres_media_h99$cc <- gage_wrapper(rlgexp, gset_go_cc, "YPDH99", "TCH99")

# 02 YPD -> TC in mar1d

In [18]:
gageres_media_mar <- list()
gageres_media_mar$bp <- gage_wrapper(rlgexp, gset_go_bp, "YPDmar1d", "TCmar1d")
gageres_media_mar$fc <- gage_wrapper(rlgexp, gset_go_fc, "YPDmar1d", "TCmar1d")
gageres_media_mar$cc <- gage_wrapper(rlgexp, gset_go_cc, "YPDmar1d", "TCmar1d")

# 03 H99 vs mar1d in YPD

In [19]:
gageres_strain_ypd <- list()
gageres_strain_ypd$bp <- gage_wrapper(rlgexp, gset_go_bp, "YPDH99", "YPDmar1d")
gageres_strain_ypd$fc <- gage_wrapper(rlgexp, gset_go_fc, "YPDH99", "YPDmar1d")
gageres_strain_ypd$cc <- gage_wrapper(rlgexp, gset_go_cc, "YPDH99", "YPDmar1d")

# 04 H99 vs mar1d in TC

In [20]:
gageres_strain_tc <- list()
gageres_strain_tc$bp <- gage_wrapper(rlgexp, gset_go_bp, "TCH99", "TCmar1d")
gageres_strain_tc$fc <- gage_wrapper(rlgexp, gset_go_fc, "TCH99", "TCmar1d")
gageres_strain_tc$cc <- gage_wrapper(rlgexp, gset_go_cc, "TCH99", "TCmar1d")

# Compare

In [22]:
tmp1 <- gageres_media_h99$bp[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp2 <- gageres_media_mar$bp[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp3 <- gageres_strain_ypd$bp[["greater"]] %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp4 <- gageres_strain_tc$bp[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)

In [28]:
tmp1

name,p.geomean,stat.mean,p.val,q.val,set.size,exp1
GO:0005975 | carbohydrate metabolic process,8.469824e-05,3.863148,8.469824e-05,0.003387929,73,8.469824e-05
GO:0055085 | transmembrane transport,3.194852e-04,3.439690,3.194852e-04,0.006389704,217,3.194852e-04
"GO:0006355 | regulation of transcription, DNA-templated",1.335641e-02,2.224637,1.335641e-02,0.141595633,185,1.335641e-02
GO:0006629 | lipid metabolic process,1.415956e-02,2.302403,1.415956e-02,0.141595633,17,1.415956e-02
GO:0007165 | signal transduction,2.531501e-02,2.038223,2.531501e-02,0.202520105,17,2.531501e-02


In [34]:
tmp4

name,p.geomean,stat.mean,p.val,q.val,set.size,exp1
GO:0005737 | cytoplasm,0.04580827,1.694553,0.04580827,0.4122744,108,0.04580827


In [24]:
print(setdiff(tmp2$name, tmp1$name) %>% sort)

[1] "GO:0008152 | metabolic process"


In [25]:
print(setdiff(tmp4$name, tmp3$name) %>% sort)

[1] "GO:0006281 | DNA repair"        "GO:0006457 | protein folding"  
[3] "GO:0007049 | cell cycle"        "GO:0008152 | metabolic process"


In [29]:
tmp1 <- gageres_media_h99$cc[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp2 <- gageres_media_mar$cc[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp3 <- gageres_strain_ypd$cc[["greater"]] %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)
tmp4 <- gageres_strain_tc$cc[["greater"]]  %>% as.data.frame %>% rownames_to_column("name") %>% filter(p.val < 0.05)

In [32]:
tmp1

name,p.geomean,stat.mean,p.val,q.val,set.size,exp1
GO:0016021 | integral component of membrane,4.015234e-06,4.498759,4.015234e-06,0.0000361371,343,4.015234e-06
GO:0016020 | membrane,1.578143e-02,2.160686,1.578143e-02,0.0710164504,142,1.578143e-02


In [33]:
tmp4

name,p.geomean,stat.mean,p.val,q.val,set.size,exp1
GO:0005737 | cytoplasm,0.04580827,1.694553,0.04580827,0.4122744,108,0.04580827


In [30]:
print(setdiff(tmp2$name, tmp1$name) %>% sort)

character(0)


In [31]:
print(setdiff(tmp4$name, tmp3$name) %>% sort)

character(0)
